### Here we make final preparation before simulation of epidemics in the city

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from shapely.geometry import Point
from tqdm import tqdm
from geopy.distance import great_circle
import time as time
import random
from scipy.stats import binom

In [2]:
city_name = 'chelyabinsk'

input_path = '../data/initial_' + city_name + '/people_' + city_name + '_splitted_workplaces.txt'

people = pd.read_csv(input_path, sep='\t', index_col=0)

def move_school_id_to_work_id(row):
    value = 0
    if row['school_id'] != -1:
        value = row['school_id']
    else:
        value = row['work_id']
    return value


people['work_id'] = people.apply(move_school_id_to_work_id, axis  = 1)
people_processed = people.drop(columns=['Unnamed: 0', 'race', 'relate'])
people_processed.head()

,sp_id,sp_hh_id,age,sex,school_id,work_id
1,790949,1,35,F,-1,1
4,293648,1,39,M,-1,1
5,293647,2,39,M,-1,1
8,721837,4,27,F,-1,1
9,344784,4,45,M,-1,1


In [3]:
def change_work_to_X(row):
    value = 0
    if (row['work_id'] == -1):
        value = 'X'
    else:
        value = row['work_id']
    return value

def change_school_to_X(row):
    value = 0
    if (row['school_id'] == -1):
        value = 'X'
    else:
        value = row['school_id']
    return value
    
    
people_processed['work_id'] = people_processed.apply(change_work_to_X, axis  = 1)
people_processed['school_id'] = people_processed.apply(change_school_to_X, axis  = 1)
people_processed.sort_index(inplace=True)

# len(people_processed[people_processed['work_id'] == 'X'])

##### And also we need to create schools.json file for model with school number as key and values are sp_id's

In [4]:
school_dict = dict()


for idx in tqdm(range(len(people_processed))):
    row = people_processed.iloc[idx]
    school_id = str(row['school_id'])
    if (school_id != 'X' and school_id != -1):
        if (school_id in school_dict):
            school_dict[school_id].append(int(idx))
        else:
            school_dict[school_id] = [int(idx)]

# del school_dict['X']

100%|██████████████████████████████| 1189525/1189525 [00:37<00:00, 31562.62it/s]


In [5]:
people_processed

,sp_id,sp_hh_id,age,sex,school_id,work_id
0,599955,1,8,F,48,48
1,790949,1,35,F,X,1
2,1094293,1,69,F,X,X
3,556704,1,2,F,X,X
4,293648,1,39,M,X,1
...,...,...,...,...,...,...
1189520,705204,472001,23,F,X,74931
1189521,955980,472001,52,F,X,74931
1189522,17939,472002,2,M,X,X
1189523,1126795,472002,72,F,X,X


In [6]:
import json

with open('../data/initial_' + city_name + '/schools.json', 'w') as fp:
    json.dump(school_dict, fp)
    
people_processed = people_processed.drop(columns=['school_id'])
people_processed.head(15)
people_processed.to_csv('../data/initial_' + city_name + '/people.txt', sep='\t')
